## 准备数据

In [47]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [48]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [49]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])          # 展平输入 [batch_size, 784]
        h = tf.nn.relu(x @ self.W1 + self.b1) # 隐藏层计算 [batch_size, 256]
        logits = h @ self.W2 + self.b2        # 输出层计算 [batch_size, 10]
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [50]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [51]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.7849474 ; accuracy 0.10425
epoch 1 : loss 2.6707208 ; accuracy 0.11365
epoch 2 : loss 2.586302 ; accuracy 0.12388334
epoch 3 : loss 2.5206647 ; accuracy 0.13425
epoch 4 : loss 2.4673584 ; accuracy 0.14303334
epoch 5 : loss 2.4225183 ; accuracy 0.152
epoch 6 : loss 2.3837411 ; accuracy 0.15948333
epoch 7 : loss 2.3494594 ; accuracy 0.16798334
epoch 8 : loss 2.3186133 ; accuracy 0.17596667
epoch 9 : loss 2.2904675 ; accuracy 0.18273333
epoch 10 : loss 2.2644846 ; accuracy 0.1904
epoch 11 : loss 2.2402673 ; accuracy 0.19915
epoch 12 : loss 2.2175083 ; accuracy 0.20715
epoch 13 : loss 2.1959739 ; accuracy 0.21505
epoch 14 : loss 2.1754775 ; accuracy 0.22343333
epoch 15 : loss 2.15587 ; accuracy 0.2316
epoch 16 : loss 2.1370304 ; accuracy 0.2395
epoch 17 : loss 2.1188598 ; accuracy 0.24743333
epoch 18 : loss 2.1012747 ; accuracy 0.25606668
epoch 19 : loss 2.0842092 ; accuracy 0.26503333
epoch 20 : loss 2.0676074 ; accuracy 0.27261665
epoch 21 : loss 2.0514235 ; accuracy 0.2